# Практика
Используемые библиотеки

In [262]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing
import os
import random

## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [226]:
#url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
#s = requests.get(url).content
#df = pd.read_csv(io.StringIO(s.decode('UTF8')))
### Если не работает загрузка on-line
df = pd.read_csv(os.path.join('data', "Химический анализ родника в Нескучном саду.csv"), sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [227]:
def transform(x):
    """
    :param x: в качестве входного параметра подается строка. Например, "в пределах 6-9"
    :return: результатом функции являетя список. Например, ['в пределах', 6, 9]
    """
    sample1 = r'([а-яЯ-а]* [а-яЯ-а]*) (\d*[-,]?\d*)'
    sample2 = r"\d*,\d*"
    sample3 = r"\d*-\d*"
    a = re.match(sample1, x)
    if re.findall(sample2, a.group(2)):
        return [a.group(1)] + [float(re.match(sample2, a.group(2))[0].replace(',', '.'))]
    elif re.findall(sample3, a.group(2)):
        return [a.group(1)] + [*map(int, re.match(sample3, a.group(2))[0].split('-'))]
    else:
        return [a.group(1)] + [int(a.group(2))]

In [228]:
transform("в пределах 6-9")

['в пределах', 6, 9]

In [229]:
def new_column():
    """
    Создает список значений новой колонки. Сравнивает показатели из столбца 'Норматив' с преобразованными значениями столбца 'Результат анализа'. Сравнение происходит без изменения исходного датафрейма
    :return: результатом является список
    """
    a = df['Результат анализа'].apply(lambda x: float(x)
                                                if x.replace('.','',1).isdigit() # убирает точку и сравнивает, число ли это
                                                else 0) # если не число, то заменяет на 0
    b = df['Норматив'].apply(transform)
    new_column = []
    for i in range(len(b)):
        if len(b[i]) == 2:
            if b[i][1] >= a[i]:
                new_column.append('Показатель в норме')
            else:
                new_column.append('Показатель превышает норму')
        else:
            if b[i][0] == 'в пределах':
                if b[i][1] <= a[i] <= b[i][2]:
                    new_column.append('Показатель в норме')
                elif a[i] <= b[i][1]:
                    new_column.append('Показатель ниже нормы')
                else:
                    new_column.append('Показатель превышает норму')
            else:
                if b[i][2] >= a[i]:
                    new_column.append('Показатель в норме')
                else:
                    new_column.append('Показатель превышает норму')
    return new_column

In [230]:
df['Вывод'] = new_column()

In [231]:
df = df.set_index('Показатель')

In [232]:
df

,Единица измерений,Результат анализа,Норматив,Вывод
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,Показатель в норме
Запах,баллы,1,не более 2-3,Показатель в норме
Цветность,градусы,б/цвета,не более 30,Показатель в норме
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,Показатель в норме
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",Показатель в норме
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",Показатель в норме
Нитраты (по NO3),мг/дм3,24,не более 45,Показатель в норме
Фосфаты (P),мг/дм3,0.36,"не более 3,5",Показатель в норме
Хлориды (Cl),мг/дм3,200,не более 350,Показатель в норме


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [282]:
count = 0
for i in range(100000):
    box = ['orange']*5 + ['apple']*4
    choice = []
    choice += [box.pop(box.index(np.random.choice(box, 1)))]
    choice += [box.pop(box.index(np.random.choice(box, 1)))]
    choice += [box.pop(box.index(np.random.choice(box, 1)))]
    if choice.count('orange') == 3:
        count += 1
print(y := count/100000, f'Результат отличается на {abs(round((y / 0.119)*100 - 100, 2))}%')


0.11706 Результат отличается на 1.63%


Результат, полученный эксперементально, очень близок к результату, полученному аналитически

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [264]:
details = ['stadart']*7 + ['non-standard']*3

In [281]:
count = 0
for i in range(200000):
    random.shuffle(details)
    if details[0] == 'non-standard' and details[1] == 'stadart':
        count += 1
print(y := count/200000, f'Результат отличается на {abs(round((y / 0.23333)*100 - 100, 2))}%')

0.23166 Результат отличается на 0.72%


Результат, полученный эксперементально, очень близок к результату, полученному аналитически